## 네이버 평점 감성분류

In [67]:
import numpy as np
import pandas as pd

### 1. 목적 
 - 네이버 영화리뷰 데이터셋을 이용해서 긍정/ 부정 분류 모델을 만들어보자
 - TF-IDF 적용해보기

### 2. 데이터 수집
 - 네이버 영화 리뷰 데이터 사용

In [68]:
df = pd.read_csv('./ratings.txt', delimiter='\t')
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


### 3. 데이터 전처리

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [70]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199992 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        199992 non-null  int64 
 1   document  199992 non-null  object
 2   label     199992 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 6.1+ MB


In [71]:
x = df['document']
y = df['label']

In [72]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [73]:
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))


149994
149994
49998
49998


In [74]:
from konlpy.tag import Okt
okt = Okt()

In [75]:
# 명사 토큰화
okt.nouns('오늘은 학교에 가는 날이에요.')

['오늘', '학교', '날']

In [76]:
def Tokenizer(text):
    return okt.nouns(text)

In [77]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

In [78]:
tfidf = TfidfVectorizer(tokenizer= Tokenizer)

In [79]:
tfidf.fit(x_train[:3])
tfidf.vocabulary_ #단어사전

c:\Users\user\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'거': 3,
 '예전': 9,
 '감독': 2,
 '하나': 12,
 '사람': 6,
 '가지': 0,
 '갈피': 1,
 '캐릭터': 10,
 '마틴': 5,
 '프리': 11,
 '널': 4,
 '사랑': 7,
 '애기': 8}

### POS tagging
- 문장을 토큰화 한 후 쪼개진 형태소에 품사를 부야하는 것

In [80]:
data = '오늘 날씨가 추워서 몸이 좋지 않다.'
okt.morphs(data)

['오늘', '날씨', '가', '추워서', '몸', '이', '좋지', '않다', '.']

In [81]:
okt.pos(data)

[('오늘', 'Noun'),
 ('날씨', 'Noun'),
 ('가', 'Josa'),
 ('추워서', 'Adjective'),
 ('몸', 'Noun'),
 ('이', 'Josa'),
 ('좋지', 'Adjective'),
 ('않다', 'Verb'),
 ('.', 'Punctuation')]

In [82]:
def Tokenizer1(text):
    df= pd.DataFrame(okt.pos(text), columns=['morph','tag'])
    df.set_index('tag', inplace=True)
    #동사, 명사, 형용사일 때만 값을 반환
    if ('Verb' in df.index)|('Adjective' in df.index)|('Noun' in df.index):
        labels = ['Verb','Adjective','Noun']
        return df.loc[df.index.intersection(labels)]['morph'].values
    else:
        return []

In [83]:
Tokenizer1(data)

array(['오늘', '날씨', '몸', '추워서', '좋지', '않다'], dtype=object)

In [84]:
tfidf = TfidfVectorizer(tokenizer= Tokenizer1)
tfidf.fit(x_train[:3])
tfidf.vocabulary_

c:\Users\user\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'거': 3,
 '예전': 15,
 '감독': 2,
 '하나': 19,
 '사람': 11,
 '가지': 0,
 '봤는데': 10,
 '하고': 18,
 '싶은말은': 13,
 '놀리지마라': 6,
 '갈피': 1,
 '캐릭터': 16,
 '못잡고': 9,
 '헤매는': 21,
 '마틴': 7,
 '프리': 17,
 '널': 5,
 '사랑': 12,
 '애기': 14,
 '먼': 8,
 '귀여워': 4,
 '해': 20}

In [85]:
x_train = x_train[:5000]
x_test = x_test[:5000]
y_train = y_train[:5000]
y_test = y_test[:5000]

In [86]:
tfidf1 = TfidfVectorizer(tokenizer= Tokenizer1)
tfidf1.fit(x_train)

c:\Users\user\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function Tokenizer1 at 0x000002AED2DE6DC0>)

In [87]:
len(tfidf1.vocabulary_) #단어 사전의 갯수 확인

12750

In [88]:
# 토큰화된 단어를 기계가 이해할 수 있도록 벡터형태로 변환
x_train = tfidf1.transform(x_train)
x_test = tfidf1.transform(x_test)

### 4. EDA

### 5~7 모델링

In [89]:
from sklearn.linear_model import LogisticRegression

In [90]:
lr =LogisticRegression()

In [91]:
lr.fit(x_train, y_train)

LogisticRegression()

In [92]:
lr.score(x_test, y_test)

0.771